In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Loading DataFrames

In [8]:
event_df = pd.read_csv('processed_df/event_df.csv')
bet_df = pd.read_csv('processed_df/betting_df.csv')

#Creating timestamps for the relevant columns
event_df["publishTime"] = pd.to_datetime(event_df['publishTime'])
bet_df["publishTime"] = pd.to_datetime(bet_df['publishTime'])

In [9]:
# Group both datasets by 'match_id'
grouped_events = event_df.groupby('match_id')
grouped_betting = bet_df.groupby('match_id')
flag = True
sample = None
eve_sam = None

# Initialize an empty list to hold the merged dataframes for each match
merged_datasets = []

# Iterate over each match_id in the events dataset
for match_id, event_group in grouped_events:
    # Find the corresponding betting group for the current match_id
    if match_id in grouped_betting.groups:
        betting_group = grouped_betting.get_group(match_id)
        
        # Perform the merge_asof operation for the current match
        merged_match_data = pd.merge_asof(event_group, betting_group, on='publishTime', direction='nearest')
        if flag:
            bet_sam = betting_group
            eve_sam = event_group
            flag = False
        # Append the result to the list of merged datasets
        merged_datasets.append(merged_match_data)

# Concatenate all the merged datasets into one dataframe
full_merged_dataset = pd.concat(merged_datasets, ignore_index=True)

In [10]:
full_merged_dataset.to_csv("processed_df/bet_event_df.csv", index= False)